## Imports
- Place %%pycodestyle at the top of any cell to check python syntax

In [1]:
%load_ext pycodestyle_magic
%load_ext autoreload
%autoreload 2

# Common imports
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import shap

# Import from my GitHub
from getxgboostmodel.getxgboostmodel import get_xgboost_model
from randomizedgridsearch.randomizedgridsearch import RandomizedGridSearch
from transformers.transformers import *

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

/Users/petermyers/Desktop/Code/commons/venv/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


ModuleNotFoundError: No module named 'getxgboostmodel'

## Initialize

In [2]:
# Example dataset
boston_data = load_boston()

# Extract pandas dataframe and target
X = pd.DataFrame(boston_data['data']).copy().values
y = pd.DataFrame(boston_data['target']).copy().values

# Train/test split
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.40, random_state=42)
test_X, final_test_X, test_y, final_test_y = train_test_split(
    test_X, test_y, test_size=0.50, random_state=42)
train_X, test_X = train_X, test_X
train_y, test_y = train_y.reshape(-1, 1), test_y.reshape(-1, 1)

# An okay model fit to the data
try:
    xgb_model
except:
    xgb_model = get_xgboost_model(train_X, train_y)

# Pipeline
pipe = Pipeline([('standard_scaler', StandardScalerTransform()),
                 ('min_max_scaler', MinMaxScalerTransform()),
                 ('binarizer', BinarizerTransform()), 
                 ('k_bins_discretizer', KBinsDiscretizerTransform()),
                 ('k_bins_discretizer2', KBinsDiscretizerTransform2()),
                 ('k_bins_discretizer3', KBinsDiscretizerTransform3()),
                 ('kernel_centerer', KernelCentererTransform()),
                 ('model', xgb_model)])

# Find the number of features
num_features = train_X.shape[1]

# Testing with these indices
indices = list(range(num_features))

# Scalers
scaler_values = [0.05]*num_features

# Possible configurations [None, True, or False] - None means not decided yet
param_distributions = {
    'standard_scaler': scaler_values,
    'min_max_scaler': scaler_values,
    'binarizer': scaler_values,
    'k_bins_discretizer': scaler_values,
    'k_bins_discretizer2': scaler_values,
    'k_bins_discretizer3': scaler_values,
    'kernel_centerer': scaler_values,
}

experiments_results = pd.DataFrame()

NameError: name 'get_xgboost_model' is not defined

## Run the Experiments

In [4]:
for iteration in range(20):
    # Randomly search the space n_iter times
    experiments_results_temp = RandomizedGridSearch(
        n_experiments=100,
        pipe=pipe,
        param_distributions=param_distributions,
        train_X=train_X,
        train_y=train_y,
        test_X=test_X,
        test_y=test_y,
        scoring='neg_mean_squared_error')
    
    # Append to experiment results
    experiments_results = experiments_results.append(experiments_results_temp, ignore_index=True)
    
    # Ignore past data
    # experiments_results = experiments_results_temp

    # Drop score
    experiments_X_df = experiments_results.drop(['score'], axis=1)

    # Get column names
    X_column_names = experiments_X_df.columns

    # Convert to numpy
    experiments_X = experiments_X_df.values
    experiments_y = experiments_results[['score']].values

    # Create an XGBoost model tuned with the experiments data
    try:
        xgb_experiments_model
        # Tune hyperparameters every once in a while
        # if iteration % 7 == 6:
        #     xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)
    except:
        xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)

    # Fit the model
    xgb_experiments_model.fit(experiments_X_df, experiments_y)

    # Extract shap values
    explainer = shap.TreeExplainer(xgb_experiments_model)
    shap_values = explainer.shap_values(experiments_X_df)

    # Shap as dataframe
    shap_values_of_experiments = pd.DataFrame(shap_values, columns=X_column_names)
    shap_values_of_experiments['score'] = experiments_y

    # Function to support analysis
    def find_significance_from_experiments_results(importance_threshold=0.001, max_toggles_to_lock_per_series=5):
        temp_df = shap_values_of_experiments.drop(['score'], axis=1).copy()
        for i in range(0, len(shap_values_of_experiments.index)):
            for j in range(0, len(shap_values_of_experiments.columns)):
                if not experiments_results.iloc[i, j]:
                    temp_df.iloc[i, j] = -1 * shap_values_of_experiments.iloc[i, j]
        temp_df = temp_df.sum().sort_values()
        options_to_set_to_false = temp_df[temp_df > 0]
        options_to_set_to_true = temp_df[temp_df < 0]    
        sum_value = (options_to_set_to_false.sum() + abs(options_to_set_to_true.sum()))
        options_to_set_to_false = options_to_set_to_false / sum_value
        options_to_set_to_true = abs(options_to_set_to_true) / sum_value
        options_to_set_to_false = options_to_set_to_false[options_to_set_to_false > importance_threshold].sort_values(ascending=False)
        options_to_set_to_true = options_to_set_to_true[options_to_set_to_true > importance_threshold]
        return options_to_set_to_false[0:max_toggles_to_lock_per_series], options_to_set_to_true[0:max_toggles_to_lock_per_series]

    # Call function
    options_to_set_to_false, options_to_set_to_true = find_significance_from_experiments_results()

    # Make the set to true DF
    options_to_set_to_true_df = pd.DataFrame()
    transformation, value = None, None
    try:
        transformation_and_value = options_to_set_to_true.keys()
    except:
        transformation, value = [], []
    if len(transformation_and_value) > 0:
        options_to_set_to_true_df["transformation"] = [x.split("__")[0] for x in transformation_and_value]
        options_to_set_to_true_df["value"] = [x.split("__")[1] for x in transformation_and_value]
    else:
        options_to_set_to_true_df["transformation"] = []
        options_to_set_to_true_df["value"] = []
    options_to_set_to_true_df["significance"] = options_to_set_to_true.values

    # Make the false DF
    options_to_set_to_false_df = pd.DataFrame()
    transformation, value = None, None
    try:
        transformation_and_value = options_to_set_to_false.keys()
    except:
        transformation, value = [], []
    if len(transformation_and_value) > 0:
        options_to_set_to_false_df["transformation"] = [x.split("__")[0] for x in transformation_and_value]
        options_to_set_to_false_df["value"] = [x.split("__")[1] for x in transformation_and_value]
    else:
        options_to_set_to_false_df["transformation"] = []
        options_to_set_to_false_df["value"] = []
    options_to_set_to_false_df["significance"] = options_to_set_to_false.values

    # Reset updates
    updates = 0
    
    # Set to True
    for index, row in options_to_set_to_true_df.iterrows():            
        if param_distributions[row['transformation']][int(row['value'])] < 1.0:
            updates += 1
        param_distributions[row['transformation']][int(row['value'])] *= 2.0        
        if param_distributions[row['transformation']][int(row['value'])] >= 1.0:
            param_distributions[row['transformation']][int(row['value'])] = 1.0        
        

    for index, row in options_to_set_to_false_df.iterrows():
        if param_distributions[row['transformation']][int(row['value'])] > 0.01:
            updates += 1        
        param_distributions[row['transformation']][int(row['value'])] *= 0.5
        if param_distributions[row['transformation']][int(row['value'])] <= 0.01:
            param_distributions[row['transformation']][int(row['value'])] = 0.01

    # End early if there is nothing to tune
    print("Updates:", updates)
    print("best score: ", min(experiments_results['score']), "\nparams: ", param_distributions)    
    if updates == 0:
        break
print("Done")

  3%|▎         | 3/100 [00:00<00:03, 26.87it/s]

Updates: 9
best score:  7.847489082977555 
params:  {'standard_scaler': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'min_max_scaler': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'binarizer': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'k_bins_discretizer': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'k_bins_discretizer2': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'k_bins_discretizer3': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025], 'kernel_centerer': [0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.2, 0.2, 0.05, 0.05, 0.1, 0.05, 0.025]}


  3%|▎         | 3/100 [00:00<00:03, 24.31it/s]

Updates: 8
best score:  7.480976061181118 
params:  {'standard_scaler': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'min_max_scaler': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'binarizer': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'k_bins_discretizer': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'k_bins_discretizer2': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'k_bins_discretizer3': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125], 'kernel_centerer': [0.05, 0.05, 0.05, 0.05, 0.025, 0.01, 0.8, 0.8, 0.05, 0.05, 0.1, 0.05, 0.0125]}


  3%|▎         | 3/100 [00:00<00:04, 24.15it/s]

...

Updates: 4
best score:  7.477899650289595 
params:  {'standard_scaler': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'min_max_scaler': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'binarizer': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'k_bins_discretizer': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'k_bins_discretizer2': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'k_bins_discretizer3': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01], 'kernel_centerer': [0.025, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.2, 0.1, 0.1, 0.01]}


  3%|▎         | 3/100 [00:00<00:04, 23.35it/s]

Updates: 4
best score:  7.477899650289595 
params:  {'standard_scaler': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'min_max_scaler': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'binarizer': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'k_bins_discretizer': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'k_bins_discretizer2': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'k_bins_discretizer3': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01], 'kernel_centerer': [0.0125, 0.05, 0.05, 0.1, 0.01, 0.01, 1.0, 1.0, 0.05, 0.4, 0.1, 0.1, 0.01]}


 60%|██████    | 60/100 [00:02<00:01, 23.38it/s]


KeyboardInterrupt: 

## Final Results

In [5]:
pd.set_option('display.max_columns', 500)

experiments_results.sort_values(by='score')

,standard_scaler__0,standard_scaler__1,standard_scaler__2,standard_scaler__3,standard_scaler__4,standard_scaler__5,standard_scaler__6,standard_scaler__7,standard_scaler__8,standard_scaler__9,standard_scaler__10,standard_scaler__11,standard_scaler__12,min_max_scaler__0,min_max_scaler__1,min_max_scaler__2,min_max_scaler__3,min_max_scaler__4,min_max_scaler__5,min_max_scaler__6,min_max_scaler__7,min_max_scaler__8,min_max_scaler__9,min_max_scaler__10,min_max_scaler__11,min_max_scaler__12,binarizer__0,binarizer__1,binarizer__2,binarizer__3,binarizer__4,binarizer__5,binarizer__6,binarizer__7,binarizer__8,binarizer__9,binarizer__10,binarizer__11,binarizer__12,k_bins_discretizer__0,k_bins_discretizer__1,k_bins_discretizer__2,k_bins_discretizer__3,k_bins_discretizer__4,k_bins_discretizer__5,k_bins_discretizer__6,k_bins_discretizer__7,k_bins_discretizer__8,k_bins_discretizer__9,k_bins_discretizer__10,k_bins_discretizer__11,k_bins_discretizer__12,k_bins_discretizer2__0,k_bins_discretizer2__1,k_bins_discretizer2__2,k_bins_discretizer2__3,k_bins_discretizer2__4,k_bins_discretizer2__5,k_bins_discretizer2__6,k_bins_discretizer2__7,k_bins_discretizer2__8,k_bins_discretizer2__9,k_bins_discretizer2__10,k_bins_discretizer2__11,k_bins_discretizer2__12,k_bins_discretizer3__0,k_bins_discretizer3__1,k_bins_discretizer3__2,k_bins_discretizer3__3,k_bins_discretizer3__4,k_bins_discretizer3__5,k_bins_discretizer3__6,k_bins_discretizer3__7,k_bins_discretizer3__8,k_bins_discretizer3__9,k_bins_discretizer3__10,k_bins_discretizer3__11,k_bins_discretizer3__12,kernel_centerer__0,kernel_centerer__1,kernel_centerer__2,kernel_centerer__3,kernel_centerer__4,kernel_centerer__5,kernel_centerer__6,kernel_centerer__7,kernel_centerer__8,kernel_centerer__9,kernel_centerer__10,kernel_centerer__11,kernel_centerer__12,score
299,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,True,False,False,7.477900
199,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,7.480976
298,False,False,False,False,False,False,True,True,False,False,True,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,7.542182
297,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,7.556167
599,Tru

In [ ]:
np.sum([False, True, True])